# 0. 导入必要的包

In [135]:
import pandas as pd
import numpy as np
import sys
sys.path.append("./tools/")
import ucTools
%load_ext autoreload

# 1. 定义必要的函数

In [48]:
def calulate_distinct_EC(eclist):
    counter =0
    res = []
    for item in eclist:
        itemArray = item.split(',')
        res+=itemArray
    res = np.array(res, dtype=object).flatten()
    res = np.unique(res)
    return res

def get_ec_list(ec_table):
    ec_list = calulate_distinct_EC(ec_table.ec_number)
    ec_list = pd.DataFrame({'ec_number':ec_list})

    ec_list['c1'] = ec_list.ec_number.map(lambda x: x.split('.')[0].strip())
    ec_list['c2'] = ec_list.ec_number.map(lambda x: x.split('.')[1].strip())
    ec_list['c3'] = ec_list.ec_number.map(lambda x: x.split('.')[2].strip())
    ec_list['c4'] = ec_list.ec_number.map(lambda x: x.split('.')[3].strip())

    return ec_list

def split_multi_function_emzyme(multi_df):
    '''
        将多功能酶拆分成多行并返回
    '''
    splitecs = []
    for index, row in multi_df.iterrows():
        ecs= row.ec_number.split(',')
        for ec in ecs:
            row.ec_number = ec
            splitecs +=[list(row.values)]
    return pd.DataFrame(splitecs,columns=multi_df.columns.values.tolist())

# 2. Sprot
## 2.1 加载数据

In [3]:
sprot = pd.read_csv('./data/sprot_with_ec.tsv', sep='\t',names=['id', 'ec_number', 'date_integraged','date_sequence_update','date_annotation_update','seq']) #读入文件
sprot.date_integraged = pd.to_datetime(sprot['date_integraged'])
sprot.date_sequence_update = pd.to_datetime(sprot['date_sequence_update'])
sprot.date_annotation_update = pd.to_datetime(sprot['date_annotation_update'])
sprot.head(3)

,id,ec_number,date_integraged,date_sequence_update,date_annotation_update,seq
0,Q6GZW6,3.6.4.-,2011-06-28,2004-07-19,2020-08-12,MDTSPYDFLKLYPWLSRGEADKGTLLDAFPGETFEQSLASDVAMRR...
1,Q6GZV6,2.7.11.1,2011-06-28,2004-07-19,2021-02-10,MATNYCDEFERNPTRNPRTGRTIKRGGPVFRALERECSDGAARVFP...
2,Q197B6,2.7.11.-,2009-06-16,2006-07-11,2020-08-12,MPLSVFAEEFAEKSVKRYIGQGLWLPCNLSDYYYYQEFHDEGGYGS...


## 2.2 统计独立的EC号， 并拆分等级

In [4]:
ec_list = get_ec_list(sprot)
ec_list.head(3)

,ec_number,c1,c2,c3,c4
0,1.-.-.-,1,-,-,-
1,1.1.1.,1,1,1,
2,1.1.1.-,1,1,1,-


## 2.3 统计每一级的EC号

In [5]:
L1 = ec_list['c1'].value_counts().sort_index()
L2 = ec_list['c2'].value_counts().sort_index()
L3 = ec_list['c3'].value_counts().sort_index()
L4 = ec_list['c4'].value_counts().sort_index()
print('Level 1 EC: {0} 个'.format(len(L1)))
print('Level 2 EC: {0} 个'.format(len(L2)))
print('Level 3 EC: {0} 个'.format(len(L3)))
print('Level 4 EC: {0} 个'.format(len(L4)))

Level 1 EC: 7 个
Level 2 EC: 25 个
Level 3 EC: 32 个
Level 4 EC: 417 个


In [6]:
sprot_ec_konwn = ec_list[(ec_list.c1 !='-') & (ec_list.c2 !='-') & (ec_list.c3!='-') & (ec_list.c4!='-')]
print('Sprot 共有：「{0}」个独立的EC NUMBER.'.format(ec_list.shape[0]))
print('Sprot 共有：「{0}」个不含未知项的EC NUMBER.'.format(sprot_ec_konwn.shape[0]))

Sprot 共有：「6312」个独立的EC NUMBER.
Sprot 共有：「6005」个不含未知项的EC NUMBER.


## 2.4 将不含未知项EC号的数据保存

In [7]:
sprot_full_ec = sprot[~sprot['ec_number'].str.contains('-')]
sprot_full_ec.to_csv('./data/sprot_full_ec.tsv',index=False, sep='\t',header=0)

# 3. Trembl

In [11]:
trembl = pd.read_csv('./data/trembl_with_ec.tsv', sep='\t',names=['id', 'ec_number', 'date_integraged','date_sequence_update','date_annotation_update','seq']) #读入文件
trembl.date_integraged = pd.to_datetime(trembl['date_integraged'])
trembl.date_sequence_update = pd.to_datetime(trembl['date_sequence_update'])
trembl.date_annotation_update = pd.to_datetime(trembl['date_annotation_update'])
trembl.head(3)

,id,ec_number,date_integraged,date_sequence_update,date_annotation_update,seq
0,O08452,3.2.1.1,01-JUL-1997,01-JUL-1997,07-OCT-2020,MNIKKLTPLLTLLLFFIVLASPVSAAKYLELEEGGVIMQAFYWDVP...
1,H9BV67,2.8.4.1,16-MAY-2012,16-MAY-2012,07-OCT-2020,AMQIGMSFIGAYKMCAGEAAVADLAFAAKHAGVIQMADILPARRAR...
2,L0AQC3,2.8.4.1,06-MAR-2013,06-MAR-2013,07-OCT-2020,AAVADLAFAAKHAGVIQMGDILPARRARGPNEPGGIKFGHFADMIQ...


## 3.2 统计独立的EC号， 并拆分等级

In [16]:
ec_list = get_ec_list(trembl)
ec_list.head(3)

,ec_number,c1,c2,c3,c4
0,1.-.-.-,1,-,-,-
1,1.1.-.-,1,1,-,-
2,1.1.1.,1,1,1,


## 3.3 统计每一级的EC号

In [17]:
L1 = ec_list['c1'].value_counts().sort_index()
L2 = ec_list['c2'].value_counts().sort_index()
L3 = ec_list['c3'].value_counts().sort_index()
L4 = ec_list['c4'].value_counts().sort_index()
print('Level 1 EC: {0} 个'.format(len(L1)))
print('Level 2 EC: {0} 个'.format(len(L2)))
print('Level 3 EC: {0} 个'.format(len(L3)))
print('Level 4 EC: {0} 个'.format(len(L4)))

Level 1 EC: 7 个
Level 2 EC: 25 个
Level 3 EC: 32 个
Level 4 EC: 407 个


In [18]:
trembl_ec_konwn = ec_list[(ec_list.c1 !='-') & (ec_list.c2 !='-') & (ec_list.c3!='-') & (ec_list.c4!='-')]
print('Sprot 共有：「{0}」个独立的EC NUMBER.'.format(ec_list.shape[0]))
print('Sprot 共有：「{0}」个不含未知项的EC NUMBER.'.format(trembl_ec_konwn.shape[0]))

Sprot 共有：「8285」个独立的EC NUMBER.
Sprot 共有：「7828」个不含未知项的EC NUMBER.


## 3.4 将不含未知项EC号的数据保存

In [19]:
trembl_full_ec = trembl[~trembl['ec_number'].str.contains('-')]
trembl_full_ec.to_csv('./data/trembl_full_ec.tsv',index=False, sep='\t', )

# 4.不同数据中EC号比对

In [219]:
in_trembl_notin_sprot = trembl_ec_konwn[~trembl_ec_konwn.ec_number.isin(sprot_ec_konwn.ec_number)]
in_sprot_notin_trembl = sprot_ec_konwn[~sprot_ec_konwn.ec_number.isin(trembl_ec_konwn.ec_number)]
print('Trembl中出现的非空EC号个数为：{0}'.format(len(trembl_ec_konwn)))
print('Sprot中出现的非空EC号个数为：{0}'.format(len(sprot_ec_konwn)))

Trembl中出现的非空EC号个数为：7828
Sprot中出现的非空EC号个数为：6005


In [220]:
sprot[sprot_ec_konwn.ec_number.isin(trembl_ec_konwn.ec_number)]

,id,ec_number,date_integraged,date_sequence_update,date_annotation_update,seq
0,Q6GZW6,3.6.4.-,2011-06-28,2004-07-19,2020-08-12,MDTSPYDFLKLYPWLSRGEADKGTLLDAFPGETFEQSLASDVAMRR...
1,Q6GZV6,2.7.11.1,2011-06-28,2004-07-19,2021-02-10,MATNYCDEFERNPTRNPRTGRTIKRGGPVFRALERECSDGAARVFP...
2,Q197B6,2.7.11.-,2009-06-16,2006-07-11,2020-08-12,MPLSVFAEEFAEKSVKRYIGQGLWLPCNLSDYYYYQEFHDEGGYGS...
3,Q6GZS1,3.6.4.-,2011-06-28,2004-07-19,2020-08-12,MAKLLRLNAIDGDMPGAGEADLFTLAPGGKAYVPFAWGSRVLGCKP...
4,Q196Y0,3.6.1.-,2009-06-16,2006-07-11,2020-08-12,MDNETSTPDIFQWCVSPFSKITLKRSMEQRDIVEFRIDATILRQIF...
...,...,...,...,...,...,...
270231,Q4R4A2,"3.4.19.12, 3.4.19.12",2006-06-27,2005-07-19,2020-12-02,MLSCDICGETVTSEPDMKAHLIVHMENEIVCPFCKLSGVSYDEMCF...
270232,Q3T9Z9,3.4.19.12,2006-06-27,2006-06-27,2021-04-07,MLSCNICGETVNSEPDMKAHLIVHMENEIICPFCKLSGINYNEICF...
270233,Q5U2S3,3.4.19.12,2006-06-27,2004-12-07,2020-12-02,MLSCDICGETVTSEPDRKAHLIVHMENEIICPFCKLSGINYNEMCF...
270234,Q621J7,2.7.11.1,2005-04-12,2004-11-23,2021-02-10,MSGGKSGTKLSSFQNLQQIGQGGFGVVYSAQRENGEKVAIKKIGNA...


In [33]:
# trembl_full_ec = pd.read_csv('./data/trembl_full_ec.tsv', sep='\t',names=['id', 'ec_number', 'date_integraged','date_sequence_update','date_annotation_update','seq'])

trembl_full_ec['label_count'] = trembl_full_ec.ec_number.map(lambda x: x.count(',') + 1)
# query['seq'].map(lambda x : len(x))

<ipython-input-33-7154929d1774>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trembl_full_ec['label_count'] = trembl_full_ec.ec_number.map(lambda x: x.count(',') + 1)


In [35]:
trembl_full_ec[trembl_full_ec.label_count>1]

,id,ec_number,date_integraged,date_sequence_update,date_annotation_update,seq,label_count
45,A0A7G9YY52,"3.5.4.41, 3.5.4.31, 3.5.4.4, 3.5.4.28",2021-02-10,2021-02-10,2021-04-07,MINMSILIKNVLLEGKERNIYIEGNLIEAISKAGNGKEAEFVIDGK...,4
66,A0A7G9YWQ6,"4.2.1.136, 5.1.99.6",2021-02-10,2021-02-10,2021-04-07,MKEEKEEALSSAEMAAIDENCKFFGLSSAQLMENAGAGIANVIKRK...,2
237,A0A2D0TC99,"1.12.1.2, 1.12.99.6",2017-12-20,2017-12-20,2021-04-07,MVKIATTWLGGCSGCHISLLDLHEELLNLLENVELVHCPVLMDVKE...,2
260,A0A2D0TCA6,"1.12.99.6, 1.12.1.2",2017-12-20,2017-12-20,2021-04-07,MVKLSVEPVTRVEGHGKISVSFDDSGNLDKVRFHVVEVRGFEKFLE...,2
308,D1JA39,"5.4.2.2, 5.4.2.8",2010-01-19,2010-01-19,2020-10-07,MHTKKREISADIFRAYDIRGIYNDDLYPETMLKIGLGLGTYAKERS...,2
...,...,...,...,...,...,...,...
32826185,E0WDN6,"2.7.7.49, 3.1.26.13",2010-11-02,2010-11-02,2020-12-02,FFRENLAFQQREARKFPSEQTRTNSPTSRECGDGGRNTLXSEAGTE...,2
32826186,B9W4E1,"2.7.7.49, 3.1.26.13",2009-03-24,2009-03-24,2020-12-02,FFRENLAFQQREARKFSSEQTRTNSPTNGEPWGGGRDNLPCETGTE...,2
32826187,H6SH45,"2.7.7.49, 3.1.26.13",2012-04-18,2012-04-18,2020-12-02,FFRENLAFQQREARKFSSEQTRAISPTSRKLWDGGRDNPLPETGTE...,2
32826188,B9W4F0,"2.7.7.49, 3.1.26.13",2009-03-24,2009-03-24,2020-12-02,FFREGLAFQQREARQFSSEQTGTNSPTSRELRDRGRDNLLSEAGTE...,2


In [21]:
sprot_full_ec = pd.read_csv('./data/sprot_full_ec.tsv', sep='\t',names=['id', 'ec_number', 'date_integraged','date_sequence_update','date_annotation_update','seq'])

## 5. EC号分析

In [49]:
# 酶与非酶数据
sprot = pd.read_hdf('./data/emzyme_noemzyme_uncertain.h5',key='data')

In [50]:
sprot_single=sprot[~sprot.isMultiFunctional]
sprot_multi = split_multi_function_emzyme(sprot[sprot.isMultiFunctional])
sprot_all = pd.concat([sprot_single, sprot_multi], axis=0)

In [59]:
len(set(sprot_all[(sprot_all.ec_specific_level==4)].ec_number))

5894

In [195]:
sprot_all

,id,name,isemzyme,isMultiFunctional,functionCounts,ec_number,ec_specific_level,date_integraged,date_sequence_update,date_annotation_update,seq,seqlength
0,P02802,MT1_MOUSE,False,False,1,-,0,1986-07-21,1986-07-21,2021-04-07,MDPNCSCSTGGSCTCTSSCACKNCKCTSCKKSCCSCCPVGCSKCAQ...,61
1,P61852,SODC_DROSI,True,False,1,1.15.1.1,4,1986-07-21,2007-01-23,2020-12-02,MVVKAVCVINGDAKGTVFFEQESSGTPVKVSGEVCGLAKGLHGFHV...,153
2,P00925,ENO2_YEAST,True,False,1,4.2.1.11,4,1986-07-21,2007-01-23,2021-04-07,MAVSKVYARSVYDSRGNPTVEVELTTEKGVFRSIVPSGASTGVHEA...,437
3,P01228,FSHB_PIG,False,False,1,-,0,1986-07-21,1991-02-01,2020-12-02,MKSLQFCFLFCCWKAICCNSCELTNITITVEKEECNFCISINTTWC...,129
4,P01227,FSHB_SHEEP,False,False,1,-,0,1986-07-21,1989-10-01,2020-12-02,MKSVQFCFLFCCWRAICCRSCELTNITITVEKEECSFCISINTTWC...,129
...,...,...,...,...,...,...,...,...,...,...,...,...
587547,Q9SYJ8,SOK1_ARATH,False,False,1,-,0,2021-04-07,2000-05-01,2021-04-07,MESNGGGGEVRRVNLVYFLSRSGHVDHPHLLRVHHLSRNGVFLRDV...,372
587548,A0A1L8HX76,WDR18_XENLA,False,False,1,-,0,2021-04-07,2017-03-15,2021-04-07,MAAPLELVLSADAAGQVANCTVWEPVTGSVLLTYRGGNTSHRGLAV...,428
587549,A0A2K1JJ00,SOK1_PHYPA,False,False,1,-,0,2021-04-07,2018-03-28,2021-04-07,MAFEENFLSAQVLYQLSWNGKLEHPHIIDVEYASDQGGLRLRDVKE...,701
587550,Q06GJ0,HEXC_OSTFU,True,False,1,3.2.1.52,4,2021-04-07,2006-10-31,2021-04-07,MWSRRIPLFIFGVLVLILSVAAEDVVWRWSCDNGKCVKLKNDPRSS...,594


In [62]:
sprot_all.reset_index(drop=True, inplace=True)

In [120]:
full_length = pd.read_csv('./data/sprot_full.tsv', sep='\t', header=None)
full_length.columns =sprot_all.columns.values.tolist()

In [121]:
sprot_all = split_multi_function_emzyme(full_length)

In [127]:
sprot_all.to_excel('./data/sprot_splited_ec_full_length_data.xlsx', index=None)

In [ ]:
sprot_all

In [139]:
uctools =  ucTools.ucTools('172.16.25.20')
cnx_mimic = uctools.db_conn()

In [203]:
SPAN=[0, 5,10, 20, 30, 40, 50, 60, 100,150, 200,250, 300,350, 400, 450,500, 600, 700, 1000, 1500, 3000, 40000]
sql = '''SELECT max(seqlength), min(seqlength) from tb_sprot;'''
MAXLEGTH = pd.read_sql_query(sql,cnx_mimic)['max'][0]

In [204]:
length_list =[]
for i in range(len(SPAN)-1):
    sql = ''' SELECT count(row_id) from tb_sprot where  seqlength BETWEEN {0} and {1}; '''.format(SPAN[i], SPAN[i+1])
    counts_value = pd.DataFrame(pd.read_sql_query(sql,cnx_mimic)).values[0][0]
    sql ='''SELECT count(*) from tb_sprot where  seqlength BETWEEN {0} and {1} and ec_number!='-';'''.format(SPAN[i], SPAN[i+1])
    has_ec = pd.DataFrame(pd.read_sql_query(sql,cnx_mimic)).values[0][0]
    sql ='''SELECT count(row_id) from tb_sprot WHERE seqlength BETWEEN {0} and {1} and ec_specific_level =4;'''.format(SPAN[i], SPAN[i+1])
    ec4 = pd.DataFrame(pd.read_sql_query(sql,cnx_mimic)).values[0][0]
    
    sql ='''SELECT count(row_id) from tb_sprot WHERE seqlength BETWEEN {0} and {1} and ec_specific_level =3;'''.format(SPAN[i], SPAN[i+1])
    ec3 = pd.DataFrame(pd.read_sql_query(sql,cnx_mimic)).values[0][0]
    
    sql ='''SELECT count(row_id) from tb_sprot WHERE seqlength BETWEEN {0} and {1} and ec_specific_level =2;'''.format(SPAN[i], SPAN[i+1])
    ec2 = pd.DataFrame(pd.read_sql_query(sql,cnx_mimic)).values[0][0]
    
    sql ='''SELECT count(row_id) from tb_sprot WHERE seqlength BETWEEN {0} and {1} and ec_specific_level =1;'''.format(SPAN[i], SPAN[i+1])
    ec1 = pd.DataFrame(pd.read_sql_query(sql,cnx_mimic)).values[0][0]
    
    sql ='''SELECT count(row_id) from tb_sprot WHERE seqlength BETWEEN {0} and {1} and ec_specific_level =0;'''.format(SPAN[i], SPAN[i+1])
    ec0 = pd.DataFrame(pd.read_sql_query(sql,cnx_mimic)).values[0][0]
    
    length_list +=[[str(SPAN[i])+'-'+str(SPAN[i+1]), int(counts_value), int(has_ec), ec4, ec3, ec2, ec1, ec0]]

In [205]:
pd.DataFrame(length_list, columns=['seqlength', 'seqcounts', 'have_ec_asigned', 'ec4', 'ec3', 'ec2', 'ec1', 'ec0'])

,seqlength,seqcounts,have_ec_asigned,ec4,ec3,ec2,ec1,ec0
0,0-5,70,5,5,0,0,0,65
1,5-10,1157,91,71,16,3,1,1066
2,10-20,2868,410,307,91,10,2,2458
3,20-30,2300,337,259,72,4,2,1963
4,30-40,4155,190,148,34,6,2,3965
5,40-50,3656,150,122,24,3,1,3506
6,50-60,5065,122,97,19,5,1,4943
7,60-100,40713,4141,3188,843,100,10,36572
8,100-150,62357,11094,8683,1507,793,111,51263
9,150-200,62484,24576,19897,2987,1298,394,37908
